# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# school_data_complete.head()
# student_data.head()

In [2]:
# Some useful methods
def format_numeric(df, clabel, vtype='dollar'):
    """
    To format the prices values
    @df : Dataframe to be formatted
    @clabel : (str) label of the column to be formated
    @vtype : (str) describing the type of value -> "dollar" / "percent"
    """
    if vtype is "dollar":
        df[clabel] = df[clabel].map("${:,.2f}".format)
    elif vtype is "percent":
        df[clabel] = df[clabel].map("{:.2f}%".format)
    elif  vtype is "integer":
        df[clabel] = df[clabel].map("{:,i}".format)
    elif vtype is "2d_float":
        df[clabel] = df[clabel].map("{:,.2f}".format)
    else:
        print('unrognized format --> df not modified')
    
    return df

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
nb_students = student_data['Student ID'].nunique()
nb_schools = school_data['School ID'].nunique()
pc_passing_math = student_data['Student ID'].loc[student_data["math_score"]>=70].count()/nb_students*100
pc_passing_reading = student_data['Student ID'].loc[student_data["reading_score"]>=70].count()/nb_students*100
# pc_passing_both = student_data['Student ID'].loc[(student_data["reading_score"]>=70) & (student_data["math_score"]>=70)].count()/nb_students*100

student_data['global_score'] = (student_data["math_score"] + student_data['reading_score'])/2

pc_passing = student_data['Student ID'].loc[student_data["global_score"]>=70].count()/nb_students*100
                                
summary = pd.DataFrame({
    'Total Schools' : [nb_schools],
    "Total Students" : [nb_students],
    "Total Budget" : [school_data['budget'].sum()],
    "Avg Math Score" : [student_data['math_score'].mean()],
    "Avg Reading Score" : [student_data['reading_score'].mean()],
    "% Passing math" :[pc_passing_math],
    "% Passing reading" : [pc_passing_reading],
    "% Overall Passing rate" : [(student_data['math_score'].mean()+student_data['reading_score'].mean())/2]
                       })

# Formating
summary = format_numeric(summary,"Total Budget")
summary = format_numeric(summary, "Avg Math Score", "2d_float")
summary = format_numeric(summary, "Avg Reading Score", "2d_float")
summary = format_numeric(summary, "% Passing math","percent")
summary = format_numeric(summary, "% Passing reading", "percent")
summary = format_numeric(summary, "% Overall Passing rate", "percent")

# Display
summary

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing math,% Passing reading,% Overall Passing rate
0,15,39170,"$24,649,428.00",78.99,81.88,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
# Creating a "super" dataframe based in the existing school_data for the summary of School key metric
# and rename a few columns
super_school_data = school_data.rename(columns={"size":"Total Students",
                                               "budget":"Total School Budget",
                                               "type": "School Type"})

# Removing the "School ID" column from the dataframe
super_school_data = super_school_data.drop(columns=['School ID'])

# Calcultating the "Per student budget" and adding it to the dataframe
super_school_data['Per student budget'] = super_school_data['Total School Budget']/super_school_data['Total Students']


In [5]:
# Grouping the complete data frame by school_name
grouped_by_school = school_data_complete.groupby(['school_name'])

# Calculating the avg score in Math and Reading of the different schools
#----------------------------------------------------------------------
avg_scores = grouped_by_school[['math_score', "reading_score"]].mean()
# We add a try/except for the merging to avoid errors when executing several times
try:
    super_school_data = super_school_data.join(avg_scores, on="school_name")
except:
    print('Already merged / or pb')
    pass
# Renaming the columns
super_school_data = super_school_data.rename(columns={"math_score":"Average Math Score",
                                 "reading_score":"Average Reading Score"})
# super_school_data.head()

In [6]:
# Calculating the passing rates
#------------------------------
# 1/ Calculating the number of students passing math and reading in each school
nb_passed = grouped_by_school['math_score','reading_score'].apply(lambda x: x[x>=70].count())
nb_passed = nb_passed.rename(columns={"math_score":"nb passing math",
                "reading_score":"nb passing reading"})
# 2/ Merging with the super dataframe
try:
    super_school_data = super_school_data.join(nb_passed, on="school_name")
except:
    print('Already merged / or pb')
    pass
# 3/ calculating the ratios
super_school_data['% Passing Math'] = super_school_data['nb passing math']/super_school_data['Total Students']*100
super_school_data['% Passing Reading'] = super_school_data['nb passing reading']/super_school_data['Total Students']*100
# 4/ Removing the nb of students passing columns
super_school_data = super_school_data.drop(columns=['nb passing math', 'nb passing reading'])

# Calculating the average passing rate
#-------------------------------------
# Shouldn't we base the ratio on the overall score (math+reading)/2 ??
super_school_data['% Overall Passing Rate'] = (super_school_data["% Passing Math"] + super_school_data['% Passing Reading'])/2

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [7]:
# Sorting in descending order
top_schools = super_school_data.sort_values(by='% Overall Passing Rate', ascending=False)

# Formating the data
top_schools = format_numeric(top_schools, 'Total School Budget')
top_schools = format_numeric(top_schools, 'Per student budget')
top_schools = format_numeric(top_schools,"Average Math Score",'2d_float')
top_schools = format_numeric(top_schools,"Average Reading Score",'2d_float')
top_schools = format_numeric(top_schools,"% Passing Math",'percent')
top_schools = format_numeric(top_schools,"% Passing Reading",'percent')
top_schools = format_numeric(top_schools,"% Overall Passing Rate",'percent')

# Display
top_schools.head()

,school_name,School Type,Total Students,Total School Budget,Per student budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
14,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [8]:
# Sorting in ascending order
bottom_schools = super_school_data.sort_values(by='% Overall Passing Rate', ascending=True)

# Formating the data
bottom_schools = format_numeric(bottom_schools, 'Total School Budget')
bottom_schools = format_numeric(bottom_schools, 'Per student budget')
bottom_schools = format_numeric(bottom_schools,"Average Math Score",'2d_float')
bottom_schools = format_numeric(bottom_schools,"Average Reading Score",'2d_float')
bottom_schools = format_numeric(bottom_schools,"% Passing Math",'percent')
bottom_schools = format_numeric(bottom_schools,"% Passing Reading",'percent')
bottom_schools = format_numeric(bottom_schools,"% Overall Passing Rate",'percent')

# Display
bottom_schools.head()

,school_name,School Type,Total Students,Total School Budget,Per student budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [9]:
def get_score_bygrade(df,grade_label,discipline):
    """
    Description: Retrieve the average score of each school the grade and the discipline
    input:
        --> df: DataFrame that contain the data
        --> grade_label: column label to get the considered grade
        --> disciption: column label to get the score to be considered
    """
    bygrade = df.loc[df["grade"]==grade_label]
    score_bygrade = bygrade.groupby("school_name")[discipline].mean()    
    return score_bygrade

In [10]:
# Building the dataFrame containing the average math score per grade per school
math_score_bygrade = pd.DataFrame({"9th":get_score_bygrade(school_data_complete, "9th", "math_score"),
                                   "10th":get_score_bygrade(school_data_complete, "10th", "math_score"),
                                  "11th":get_score_bygrade(school_data_complete, "11th", "math_score"),
                                  "12th":get_score_bygrade(school_data_complete, "12th", "math_score")})

# Formating
math_score_bygrade = format_numeric(math_score_bygrade,"9th","2d_float")
math_score_bygrade = format_numeric(math_score_bygrade,"10th","2d_float")
math_score_bygrade = format_numeric(math_score_bygrade,"11th","2d_float")
math_score_bygrade = format_numeric(math_score_bygrade,"12th","2d_float")

# Display
math_score_bygrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
# Building the dataFrame containing the average reading score per grade per school
reading_score_bygrade = pd.DataFrame({"9th":get_score_bygrade(school_data_complete, "9th", "reading_score"),
                                 "10th":get_score_bygrade(school_data_complete, "10th", "reading_score"),
                                 "11th":get_score_bygrade(school_data_complete, "11th", "reading_score"),
                                 "12th":get_score_bygrade(school_data_complete, "12th", "reading_score")})

# Formating
reading_score_bygrade = format_numeric(reading_score_bygrade,"9th","2d_float")
reading_score_bygrade = format_numeric(reading_score_bygrade,"10th","2d_float")
reading_score_bygrade = format_numeric(reading_score_bygrade,"11th","2d_float")
reading_score_bygrade = format_numeric(reading_score_bygrade,"12th","2d_float")

# Display
reading_score_bygrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [13]:
# Cutting the School dataFrame into spending ranges
super_school_data['Spending Ranges'] = pd.cut(super_school_data['Per student budget'], spending_bins, labels=group_names)

# Grouping by the spending Ranges
grouped_by_spendingCat = super_school_data.groupby("Spending Ranges")

# Building the summary dataframe containing the score by spending range
score_by_spending = pd.DataFrame({"Average Math Score": grouped_by_spendingCat['Average Math Score'].mean(),
                      "Average Reading Score": grouped_by_spendingCat['Average Reading Score'].mean(), 
                    "% Passing Math": grouped_by_spendingCat['% Passing Math'].mean(),
                       "% Passing Reading": grouped_by_spendingCat['% Passing Reading'].mean(),
                       "% Overall Passing Rate": grouped_by_spendingCat['% Overall Passing Rate'].mean(),
                      })

# Formating
score_by_spending = format_numeric(score_by_spending,"Average Math Score", "2d_float" )
score_by_spending = format_numeric(score_by_spending,"Average Reading Score", "2d_float" )
score_by_spending = format_numeric(score_by_spending,"% Passing Math", "percent" )
score_by_spending = format_numeric(score_by_spending,"% Passing Reading", "percent" )
score_by_spending = format_numeric(score_by_spending,"% Overall Passing Rate", "percent" )

# Display
score_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges,,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [15]:
# Cutting the School dataFrame into spending ranges
super_school_data['Size Ranges'] = pd.cut(super_school_data['Total Students'], size_bins, labels=group_names)

# Grouping by the spending Ranges
grouped_by_sizeCat = super_school_data.groupby("Size Ranges")

# Building the summary dataframe containing the score by spending range
score_by_size = pd.DataFrame({"Average Math Score": grouped_by_sizeCat['Average Math Score'].mean(),
                           "Average Reading Score": grouped_by_sizeCat['Average Reading Score'].mean(), 
                                  "% Passing Math": grouped_by_sizeCat['% Passing Math'].mean(),
                               "% Passing Reading": grouped_by_sizeCat['% Passing Reading'].mean(),
                          "% Overall Passing Rate": grouped_by_sizeCat['% Overall Passing Rate'].mean(),
                      })

# Formating
score_by_size = format_numeric(score_by_size,"Average Math Score", "2d_float" )
score_by_size = format_numeric(score_by_size,"Average Reading Score", "2d_float" )
score_by_size = format_numeric(score_by_size,"% Passing Math", "percent" )
score_by_size = format_numeric(score_by_size,"% Passing Reading", "percent" )
score_by_size = format_numeric(score_by_size,"% Overall Passing Rate", "percent" )

# Display
score_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Size Ranges,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [16]:
# Grouping by the spending Ranges
grouped_by_type = super_school_data.groupby("School Type")

# Building the summary dataframe containing the score by spending range
score_by_type = pd.DataFrame({"Average Math Score": grouped_by_type['Average Math Score'].mean(),
                           "Average Reading Score": grouped_by_type['Average Reading Score'].mean(), 
                                  "% Passing Math": grouped_by_type['% Passing Math'].mean(),
                               "% Passing Reading": grouped_by_type['% Passing Reading'].mean(),
                          "% Overall Passing Rate": grouped_by_type['% Overall Passing Rate'].mean(),
                      })

# Formating
score_by_type = format_numeric(score_by_type,"Average Math Score", "2d_float" )
score_by_type = format_numeric(score_by_type,"Average Reading Score", "2d_float" )
score_by_type = format_numeric(score_by_type,"% Passing Math", "percent" )
score_by_type = format_numeric(score_by_type,"% Passing Reading", "percent" )
score_by_type = format_numeric(score_by_type,"% Overall Passing Rate", "percent" )

# Display
score_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
